In [1]:
using ITensors

# Creating Number States in ITensor

In [29]:
sites = siteinds("Qudit", 10; dim=10);

In [30]:
mps = randomMPS(sites; linkdims=4)

MPS
[1] ((dim=10|id=507|"Qudit,Site,n=1"), (dim=4|id=821|"Link,l=1"))
[2] ((dim=4|id=821|"Link,l=1"), (dim=10|id=627|"Qudit,Site,n=2"), (dim=4|id=808|"Link,l=2"))
[3] ((dim=4|id=808|"Link,l=2"), (dim=10|id=996|"Qudit,Site,n=3"), (dim=4|id=474|"Link,l=3"))
[4] ((dim=4|id=474|"Link,l=3"), (dim=10|id=101|"Qudit,Site,n=4"), (dim=4|id=800|"Link,l=4"))
[5] ((dim=4|id=800|"Link,l=4"), (dim=10|id=203|"Qudit,Site,n=5"), (dim=4|id=851|"Link,l=5"))
[6] ((dim=4|id=851|"Link,l=5"), (dim=10|id=63|"Qudit,Site,n=6"), (dim=4|id=80|"Link,l=6"))
[7] ((dim=4|id=80|"Link,l=6"), (dim=10|id=530|"Qudit,Site,n=7"), (dim=4|id=365|"Link,l=7"))
[8] ((dim=4|id=365|"Link,l=7"), (dim=10|id=120|"Qudit,Site,n=8"), (dim=4|id=999|"Link,l=8"))
[9] ((dim=4|id=999|"Link,l=8"), (dim=10|id=972|"Qudit,Site,n=9"), (dim=4|id=218|"Link,l=9"))
[10] ((dim=4|id=218|"Link,l=9"), (dim=10|id=187|"Qudit,Site,n=10"))


In [31]:
mps2 = randomMPS(sites; linkdims=4)

MPS
[1] ((dim=10|id=507|"Qudit,Site,n=1"), (dim=4|id=141|"Link,l=1"))
[2] ((dim=4|id=141|"Link,l=1"), (dim=10|id=627|"Qudit,Site,n=2"), (dim=4|id=894|"Link,l=2"))
[3] ((dim=4|id=894|"Link,l=2"), (dim=10|id=996|"Qudit,Site,n=3"), (dim=4|id=333|"Link,l=3"))
[4] ((dim=4|id=333|"Link,l=3"), (dim=10|id=101|"Qudit,Site,n=4"), (dim=4|id=946|"Link,l=4"))
[5] ((dim=4|id=946|"Link,l=4"), (dim=10|id=203|"Qudit,Site,n=5"), (dim=4|id=394|"Link,l=5"))
[6] ((dim=4|id=394|"Link,l=5"), (dim=10|id=63|"Qudit,Site,n=6"), (dim=4|id=799|"Link,l=6"))
[7] ((dim=4|id=799|"Link,l=6"), (dim=10|id=530|"Qudit,Site,n=7"), (dim=4|id=937|"Link,l=7"))
[8] ((dim=4|id=937|"Link,l=7"), (dim=10|id=120|"Qudit,Site,n=8"), (dim=4|id=102|"Link,l=8"))
[9] ((dim=4|id=102|"Link,l=8"), (dim=10|id=972|"Qudit,Site,n=9"), (dim=4|id=990|"Link,l=9"))
[10] ((dim=4|id=990|"Link,l=9"), (dim=10|id=187|"Qudit,Site,n=10"))
